As a first step, we have to import packages, needed for data analysis.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

We got our packages imported, now we want to take a look at our data and inspect them.

In [2]:
raw_data = pd.read_csv("netflix_titles.csv")


raw_data.info

<bound method DataFrame.info of      show_id     type                  title         director  \
0         s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1         s2  TV Show          Blood & Water              NaN   
2         s3  TV Show              Ganglands  Julien Leclercq   
3         s4  TV Show  Jailbirds New Orleans              NaN   
4         s5  TV Show           Kota Factory              NaN   
...      ...      ...                    ...              ...   
8802   s8803    Movie                 Zodiac    David Fincher   
8803   s8804  TV Show            Zombie Dumb              NaN   
8804   s8805    Movie             Zombieland  Ruben Fleischer   
8805   s8806    Movie                   Zoom     Peter Hewitt   
8806   s8807    Movie                 Zubaan      Mozez Singh   

                                                   cast        country  \
0                                                   NaN  United States   
1     Ama Qamata, Khosi Ngema, Gail Mab

From .info, we see that we have dataset of size 8807 x 12, which contains NaN values. We want to deep diver and inspect their distribution, to choose optimal way of processing them. 

In [31]:
raw_data.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

After seeing that vast majority of missing values is contained in "director" column, we are going to fill them, using cast.

In [3]:
auxiliary_df = raw_data[["director","cast"]]

auxiliary_df["cast_list"] = auxiliary_df["cast"].str.split(",")

auxiliary_df_2 = auxiliary_df.cast_list.apply(pd.Series).fillna("")

auxiliary_df_2["director"] = auxiliary_df["director"]

auxiliary_df_2.dropna()

/var/folders/td/68d_dckd4_9d0fs13g_7wv2c0000gn/T/ipykernel_17572/2096268624.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  auxiliary_df["cast_list"] = auxiliary_df["cast"].str.split(",")


,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,director
0,,,,,,,,,,,...,,,,,,,,,,Kirsten Johnson
2,Sami Bouajila,Tracy Gotoas,Samuel Jouy,Nabiha Akkari,Sofia Lesaffre,Salim Kechiouche,Noureddine Farihi,Geert Van Rampelberg,Bakary Diombera,,...,,,,,,,,,,Julien Leclercq
5,Kate Siegel,Zach Gilford,Hamish Linklater,Henry Thomas,Kristin Lehman,Samantha Sloyan,Igby Rigney,Rahul Kohli,Annarah Cymone,Annabeth Gish,...,,,,,,,,,,Mike Flanagan
6,Vanessa Hudgens,Kimiko Glenn,James Marsden,Sofia Carson,Liza Koshy,Ken Jeong,Elizabeth Perkins,Jane Krakowski,Michael McKean,Phil LaMarr,...,,,,,,,,,,"Robert Cullen, José Luis Ucha"
7,Kofi Ghanaba,Oyafunmike Ogunlano,Alexandra Duah,Nick Medley,Mutabaruka,Afemo Omilami,Reggie Carter,Mzuri,,,...,,,,,,,,,,Haile Gerima
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8801,Ali Suliman,Saleh Bakri,Yasa,Ali Al-Jabri,Mansoor Alfeeli,Ahd,,,,,...,,,,,,,,,,Majid Al Ansari
8802,Mark Ruffalo,Jake Gyllenhaal,Robert Downey Jr.,Anthony Edwards,Brian Cox,Elias Koteas,Donal Logue,John Carroll Lynch,Dermot Mulroney,Chloë Sevigny,...,,,,,,,,,,David Fincher
8804,Jesse Eisenberg,Woody Harrelson,Emma Stone,Abigail Breslin,Amber Heard,Bill Murray,Derek Graf,,,,...,,,,,,,,,,Ruben Fleischer
8805,Tim Allen,Courteney Cox,Chevy Chase,Kate Mara,Ryan Newman,Michael Cassidy,Spencer Breslin,Rip Torn,Kevin Zegers,,...,,,,,,,,,,Peter Hewitt
